In [ ]:
import poppy
import astropy.units as u
import logging
import importlib
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
import numpy as np
%matplotlib inline
plt.rcParams.update({"font.size":8})
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)



Starting with Maxwell's equations:
* Faraday: $\nabla \times \vec{E} = \frac{-\partial \vec{B}}{\partial t}$
* Ampere-Maxwell: $\nabla \times \vec{B} =\mu_o \epsilon_o\frac{\partial \vec{E}}{\partial t}+\mu_o \vec{J}$

Taking the curle of Faraday's law gives:

$\nabla \times (\nabla \times \vec{E}) = \frac{-\partial \vec{B}}{\partial t} =$
$\mu_o \epsilon_o\frac{\partial^2 \vec{E}}{\partial t^2}$

Taking advantage of the vector identity for a vector $\vec{X}$:

$\nabla \times (\nabla \times \vec{X}) = \nabla(\nabla \cdot \vec{X}) - \nabla^2\vec{X}$

and  Gauss' law for a wave in a vacuum ($\nabla\cdot\vec{E}=0$), we find the wave equation for the vector electric field:

$-\nabla^2\vec{E} = \mu_o \epsilon_o\frac{\partial^2 \vec{E}}{\partial t^2} $

To derive diffraction theory we will make a series of approximations.



Approx. 1) the scalar wave equation, which requires an approximately homogeneous medium. Each component of the wave equation can be expressed as a separate wave equation:

$\nabla^2E_x - \frac{n^2}{c^2}\frac{\partial^2E_x}{\partial t^2}=0$,

Any scalar component ($u$) follows the same equation:
 
$\nabla^2u - \frac{n^2}{c^2}\frac{\partial^2u}{\partial t^2}=0$.

Assuming a time-independent solution, this equation becomes the Helmoltz Equation ([Wikipedia](https://en.wikipedia.org/wiki/Helmholtz_equation), [Wolfram Mathworld](http://mathworld.wolfram.com/HelmholtzDifferentialEquation.html)):

$(\nabla^2 +(\frac{2\pi}{\lambda})^2)U=(\nabla^2 +k^2)U=0$

Plane wave solution:

$p(x,y,z,t) = e^{j(\vec{k}\cdot\vec{r}-2\pi\nu t)}$

Fourier transforms:



Approx. 2) Paraxial solution, where the angle between $k$ and the optical axis $z$ is small. 

plugging in the plane wave solution

$u(P,t)= Re\{U(P)e^{-i2\pi\nu t}\}$

where the phasor is composed of the wave amplitude and phase: $U(P)=A(p)e^{i\phi(P)}$

### Action of a Lens

As detailed subsequently, in the paraxial approximation, the operation of a lens is to apply a quadratic phase function, which is equivalent to a parabolic surface.

The action of a lens is to change the phase as a function of radius. 
Neglecting propagation between the curved surface 
For a lens with a spherical surface radius R, the optical path difference in the $x,y$ plane is 

$\Delta (x,y) = n(C \pm \left(R-\sqrt{R^2-x^2-y^2}\right))$
$ = n(C \pm R\left(1-\sqrt{1-\frac{x^2-y^2}{R^2}}\right))$.

Where C is a constant depending on the total thickness of the lens and the sign depends on the the orientation of the surface relative to the incident beam.

In the paraxial case, the relation above becomes:

$\Delta (x,y) = C \pm R\left({\frac{x^2-y^2}{2R}}\right)$.

for lens with two radii of curvature, the total thickness is:

$\Delta (x,y) = C - \frac{x^2-y^2}{2}\left(\frac{1}{R_1}-\frac{1}{R_2}\right)$.

Expressing the focal length as the thin lensmakers equation:

$\frac{1}{f}\equiv (n-1)\left(\frac{1}{R_1} -\frac{1}{R_2}\right)$.

$\Delta (x,y) = C - \frac{x^2-y^2}{2}\frac{1}{(n-1)f}$.

the phase difference due to glass lens of index of refraction $n$ is:

$\Delta\phi(x,y) = kn\Delta(x,y) + k(\Delta_0-\Delta(x,y))$

The action of the lens on an incident wavefront can be expressed as a complex transformation of:

$e^{jk\Delta_0}e^{jk(n-1)\Delta(x,y)} = e^{-j\frac{k}{2f}(x^2-y^2)}$ 

This is the same quadratic phase factor as far-field propagation and a parabolic optic.

Thus, the transformation of a lens from a plane-wave to a focal plane is the same Fraunhofer far-field propagation.

(Goodman eq. 5-1).

The propagation approach use by POPPY, described in Lawrence 1992, defines the wavefront state in terms of inside and outside the Rayleigh range. 
Inside is defined as planar, and outside is defined as spherical. 

Propagation between any two planes is carried out by first determining whether the source and target plane will be spherical. 

The sequence employed in the POPPY propagation code is:

1. Initiate a plane wave.

3. Determine the effective wavefront curvature, based on the current wavefront curvature incident on the lens and the lens radius of curvature. 
4. 


References:

Goodman 2005

Lawrence 1992.

In [ ]:
#define some parameters of the Hubble Space Telescope 
diam = 2.4 * u.m
fl_pri = 5.52085 * u.m
d_pri_sec = 4.907028205 * u.m
fl_sec = -0.6790325 * u.m
d_sec_to_focus = 6.3919974 * u.m

### Initiate a plane wave 

and apply a circular aperture.

In [ ]:
wf_init = poppy.FresnelWavefront(2.4*u.m,
    wavelength=500*u.nm,
    npix=1024,
    oversample=2)
wf_init.display(what="both")

In [ ]:
wf_init *= poppy.CircularAperture(radius=1.2* u.m)
wf_init.display(what="both")

### Multiply the wavefront by a lens operator

Multiplication of the wavefront by a Quadradic lens OpticalElement initiates a sequence of calculations:

#### 1. Determine if the incident beam is spherical or planar.

If the input beam is planar (or the distance to the waist is less than the rayleigh distance), the radius of curvature of the input beam $r_i=\infty$ and output beam is the focal length $r_{o}=-f$.



In [ ]:
fl_pri = 5.52085 * u.m
wf=wf_init.copy()
M1 = poppy.QuadraticLens(fl_pri, name='Lens1')
plt.figure(figsize=[7,3])
plt.subplot(121)
plt.imshow(wf.phase,cmap=plt.cm.magma)
plt.colorbar()
plt.title("Initial Wavefront Phase")
wf *= M1

#wf.display(what="phase",colorbar=True,vmax=0.01,vmin=-.01)
plt.subplot(122)
plt.imshow(wf.phase,cmap=plt.cm.magma)
plt.colorbar()
plt.title("Phase at M1")
plt.tight_layout()

In [ ]:
#wf.display(what="phase",colorbar=True,vmax=0.01,vmin=-.01)
plt.figure(figsize=[4,3])

plt.imshow(wf.phase,cmap=plt.cm.magma)
plt.colorbar()
plt.title("Phase at M1")

#### 2. Calculate the radius of curvature of the guassian beam $r_c$:
If the input beam in not planar, then the lens operates on a curved wavefront and 

$\frac{1}{r_o} = \frac{1}{r_c}-\frac{1}{f}$

$r_c= \delta_z\left(1+(\frac{z_r}{\delta z})^2\right)$ 
Where $\delta z$ is the distance to the waist $\delta z= z-z_{w_0}$

For example, the beam incident on the HST secondary is curved, as can be see from the wavefront map below and the radius of curvature of the incident wave of -.614 meters.
 

In [ ]:
m2 = poppy.QuadraticLens(fl_sec, name='Secondary')
wf.propagate_fresnel(d_pri_sec)
plt.figure(figsize=[8,3])
plt.subplot(121)
plt.imshow(wf.phase*wf.intensity,
           cmap=plt.cm.BrBG,
           norm=SymLogNorm(2e-7,-2e-7))#,vmin=-20e-8)#,vmin=-100e-6)
plt.colorbar()
plt.title("Phase incident on M2 @ z="+str(np.round(wf.z,3))+
         ",rc="+str(np.round(wf.r_c(),3)))

plt.subplot(122)
wf*=m2
plt.imshow(wf.phase*wf.intensity,
           cmap=plt.cm.BrBG,
           norm=SymLogNorm(2e-7,-2e-7))#,vmin=-20e-8)#,vmin=-100e-6)
plt.title("Phase departing M2 @ z="+str(np.round(wf.z,3))+
         ",rc="+str(np.round(wf.r_c(),3)))


plt.tight_layout()



In [ ]:
#### 3.

In [ ]:

plt.subplot(121)
#wf*=m2
plt.imshow(wf.phase*wf.intensity,
           cmap=plt.cm.BrBG,
           norm=SymLogNorm(2e-7,-2e-7))#,vmin=-20e-8)#,vmin=-100e-6)
plt.title("Phase departing M2 @ z="+str(np.round(wf.z,3))+
         ",rc="+str(np.round(wf.r_c(),3)))
wf.propagate_fresnel(-wf.r_c())

plt.tight_layout()



In [ ]:
plt.imshow(wf.intensity,
           cmap=plt.cm.plasma,
           norm=SymLogNorm(2e-7,-2e-7))#,vmin=-20e-8)#,vmin=-100e-6)